In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
sub = 'pilot-53'
ses = 'ses-01'
battery = 'camcan'
task = 'cattell'

main_dir = '/home/sshankar/ibc/analysis_pipeline/ibc_main/neurospin_data/info/'
datadir = os.path.join(main_dir, sub, battery)
csv_file = 'extracted_' + task + '_sub-53_' + ses + '.csv'
tsv_file = task + '_sub-53_' + ses + '.tsv'

In [3]:
file_df = pd.read_csv(os.path.join(datadir, csv_file))

In [4]:
# In the csv file extracted from the E-Prime txt file, 
# run onset is the 'GetReady.OffsetTime' column of last row, 
# and all other columns of this row are empty.
run_start = (file_df.loc[~np.isnan(file_df['GetReady.OffsetTime'])])['GetReady.OffsetTime']
run_start = run_start.values[0]

In [7]:
# Trial onsets are in 'CatImage.OnsetTime'
trial_onsets = (file_df.loc[~np.isnan(file_df['CatImage.OnsetTime'])])['CatImage.OnsetTime']

# Trial offsets are in 'CatImage.OffsetTime'
trial_offsets = (file_df.loc[~np.isnan(file_df['CatImage.OffsetTime'])])['CatImage.OffsetTime']

# In this task, trial duration and RT are the same
durations = trial_offsets - trial_onsets

# Finally, trial_types are in 'Running'
trial_types = (file_df.loc[~np.isnan(file_df['CatImage.OffsetTime'])])['Running']
trial_types[trial_types=='CatEasy'] = 'easy'
trial_types[trial_types=='CatHard'] = 'hard'

/home/sshankar/env/fastsrm/lib/python3.6/site-packages/pandas/core/generic.py:5191: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  object.__getattribute__(self, name)
/home/sshankar/env/fastsrm/lib/python3.6/site-packages/pandas/core/generic.py:5192: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  return object.__setattr__(self, name, value)


In [6]:
# Responses are in 'CatImage.RESP'. This field is not used in tsv file
# generation but is useful for behavioral analysis.
responses = (file_df.loc[~np.isnan(file_df['CatImage.OnsetTime'])])['CatImage.RESP']

In [9]:
# Quick analysis to look at mean RTs for easy and difficult trials
# easy RTs < difficult RTs

# Find the indices of the easy trials
easy = trial_types == 'CatEasy'

# extract all the RTs
RTs = trial_offsets - trial_onsets

# Calculate mean RTs
easy_mean = np.mean(RTs[easy])
difficult_mean = np.mean(RTs[~easy])

print(easy_mean, difficult_mean)

616.5436893203884 1929.0816326530612


In [6]:
# Create a DataFrame to save to events file
df = pd.DataFrame({'onset':(trial_onsets-run_start)/1000, 'duration':durations/1000, 'trial_type':trial_types})
df.to_csv(os.path.join(datadir, tsv_file), sep='\t', float_format='%0.3f', index=False)